In [ ]:
import sys
import subprocess
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction
from bayes_opt import SequentialDomainReductionTransformer
bounds_transformer = SequentialDomainReductionTransformer()
import matplotlib as mtp
mtp.rcParams.update({'font.size':16})
import numpy as np
from random import *
import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib inline
seed(5000)
factor = 5000 #Why?

In [ ]:
"""def get_runtime(x): # or make a call to FaaS platform to get runtime or cost 
    pars = [2.67347860e+00, 2.18887259e+01, 5.49402930e-03]
    return (pars[0]+pars[1]*(1-pars[2])**(x-128))/factor"""

def get_exectime(memory, cpu):
    return

"""def get_lambda_cost (memory, exec_time): # calculate AWS Lambda cost, given memory allocated and execution time
    return 0.0000002+(memory/1024.0)*exec_time*0.00001667"""

def get_gcf_cost (memory, cpu, exec_time):
    return

In [ ]:
x = np.linspace(128, 32768, 33000-128).reshape(-1, 1)
y = np.linspace(0.083, 8, 800 - 8).reshape(-1, 1)
# node_type = np.linspace(0, 10000, 2).reshape(-1, 1)
z_c = get_exectime(x, y)*factor
plt.plot(x, y, z_c, label='Ground Truth')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# define exploration space i.e. memory values 
pbounds = { 'x': (128, 32768), 'y': (0.083, 8)}

optimizer = BayesianOptimization(
    f=None,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=randint(128, 32768) # seed를 128, 3008로 잡은 이유가 있는지?
)

# BO parameters 

acq_type = "ei"
xi_value = 0.001 # more exploration 
kappa_value = 10.0
alpha_value = 0.1
utility = UtilityFunction(kind=acq_type, kappa = 10, xi=xi_value)


In [ ]:
def posterior(optimizer, x_obs, y_obs, z_obs, grid):
    optimizer._gp.fit(x_obs, y_obs)

    mu, sigma = optimizer._gp.predict(grid, return_std=True)
    return mu, sigma


def plot_gp(optimizer, x, y, z_c, acq_type, xi_value, kappa_value):
    global graph_counter
    fig = plt.figure(figsize=(7, 7, 7))
    steps = len(optimizer.space)
    gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
    axis = plt.subplot(gs[0])    
    plt.subplots_adjust(wspace=0, hspace=0.12)
    x_obs = np.array([[res["params"]["x"]] for res in optimizer.res])
    y_obs = np.array([[res["params"]["y"]] for res in optimizer.res])
    z_obs = np.array([res["target"] for res in optimizer.res])
    mu, sigma = posterior(optimizer, x_obs, y_obs, z_obs, x)
    # axis.plot(x, get_runtime(x)*factor, linewidth=3, label='Measured')
    # axis.plot(x, mu*factor, '--', color='k', label='Prediction')
    # axis.plot(x_obs.flatten(), get_runtime(x_obs)*factor, 'D', markersize=8, label=u'Observations', color='r')
    # axis.fill_between(x.flatten(), (mu + sigma)*factor, (mu - sigma)*factor, alpha=0.3)

    axis.set_xlim((128, 32768))
    axis.set_ylim((0.083, 8))
    axis.set_zlim((0, 40))
    axis.set_xlabel('Memory (MB)', fontdict={'size':20})
    axis.set_ylabel('vCPU', fontdict={'size':20})
    axis.set_zlabel('Execution Time (ms)', fontdict={'size':20})
    z_max = optimizer.max['target']
    axis.legend(loc=1, bbox_to_anchor=(0.77, 0.99), borderaxespad=0., ncol=2, prop={'size': 14})
    return mu,sigma

In [ ]:
# BO in action ...
counter = 1
total_samples = 10 

while True:
    
    next_point = optimizer.suggest(utility)
    next_point['x'] = int(round(float(next_point['x']))/ float(64)) * 64
    next_point['y'] = float(next_point['y'])
    print("Next point to probe is:", next_point)
    # target = get_runtime(**next_point)
    target = get_exectime(**next_point)
    print("Found the target value to be:", target)
    next_point['x'] = next_point['x'] + random()
    next_point['y'] = next_point['y'] + random()
    optimizer.register(params=next_point, target=target)
    optimizer.set_gp_params(alpha=alpha_value)
    optimizer.set_gp_params(normalize_z=True)

    mu, sigma = plot_gp(optimizer, x, y, z_c, acq_type, xi_value, kappa_value)
    print ('Samples so far:', counter)
    counter+=1
    if counter > total_samples:
        break
    
print("BEST:", optimizer.max)


In [ ]:
##plot grounth truth and prediction

In [ ]:
predicted_runtimes =  mu*factor

plt.plot(x, get_runtime(x)*factor, linewidth=3, label='ground truth')
plt.plot(x,predicted_runtimes, '--', color='k', label='Prediction')
plt.xlabel('memory (MB)')
plt.ylabel('runtime (s)')
plt.legend()
